In [ ]:
import pandas as pd
import numpy as np

matrix_df = pd.read_csv("MATRIX.csv", header=None).astype(int)

orientation = 0
rowdata = []

with open("/content/INPUT.txt", "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith("ORIENTATION:"):
            orientation = int(line.split(":")[1].strip())
        elif line.startswith("ROWDATA:"):
            values = line.replace("ROWDATA:", "").strip().split()
            rowdata.append(values)

#MILESTONE 1
ink_df = pd.DataFrame(rowdata)

def rotate_back_to_0(df, orientation):
    array = df.values
    if orientation == 90:
        array = np.rot90(array, k=3)
    elif orientation == 180:
        array = np.rot90(array, k=2)
    elif orientation == 270:
        array = np.rot90(array, k=1)
    return pd.DataFrame(array)

ink_df = rotate_back_to_0(ink_df, orientation)

min_rows = min(matrix_df.shape[0], ink_df.shape[0])
min_cols = min(matrix_df.shape[1], ink_df.shape[1])

matrix_df = matrix_df.iloc[:min_rows, :min_cols].reset_index(drop=True)
ink_df = ink_df.iloc[:min_rows, :min_cols].reset_index(drop=True)

for i in range(min_rows):
    for j in range(min_cols):
        if matrix_df.iat[i, j] == 0:
            ink_df.iat[i, j] = "000"

good_count = (ink_df == "100").sum().sum()
bad_count = (ink_df == "005").sum().sum()

output_filename = "M1T2.txt"
with open(output_filename, "w") as f:
    f.write(f"NO OF GOOD DIES:{good_count}\n")
    f.write(f"NO OF BAD DIES:{bad_count}\n")
    for row in ink_df.itertuples(index=False):
        f.write("ROWDATA: " + " ".join(row) + "\n")

print(f"Done {output_filename}")

Done! Output written to M1T2.txt


In [ ]:
import pandas as pd
import numpy as np

matrix_df = pd.read_csv("MATRIX.csv", header=None).astype(int)
orientation = 0
distance = 1
die_height = 1
die_width = 1
good_code = bad_code = inactive_code = "100"
rowdata = []
with open("INPUT.txt", "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith("ORIENTATION:"):
            orientation = int(line.split(":")[1].strip())
        elif line.startswith("DISTANCE:"):
            distance = int(line.split(":")[1].strip())
        elif line.startswith("DIESIZE:"):
            parts = line.split(":")[1].strip().split(",")
            die_height, die_width = int(parts[0]), int(parts[1])
        elif line.startswith("GOOD_DIE:"):
            good_code = line.split(":")[1].strip()
        elif line.startswith("ACTIVE_BAD_DIE:"):
            bad_code = line.split(":")[1].strip()
        elif line.startswith("INACTIVE_BAD_DIE:"):
            inactive_code = line.split(":")[1].strip()
        elif line.startswith("ROWDATA:"):
            row = line.replace("ROWDATA:", "").split()
            rowdata.append(row)
ink_df = pd.DataFrame(rowdata)
arr = ink_df.values
if orientation == 90:
    arr = np.rot90(arr, k=3)
elif orientation == 180:
    arr = np.rot90(arr, k=2)
elif orientation == 270:
    arr = np.rot90(arr, k=1)
ink_df = pd.DataFrame(arr)
rows, cols = min(matrix_df.shape[0], ink_df.shape[0]), min(matrix_df.shape[1], ink_df.shape[1])
matrix_df = matrix_df.iloc[:rows, :cols].reset_index(drop=True)
ink_df = ink_df.iloc[:rows, :cols].reset_index(drop=True)

#raw_ink = ink_df.copy()

for i in range(rows):
    for j in range(cols):
        if matrix_df.iat[i, j] == 0:
            ink_df.iat[i, j] = "000"


v_range=distance//die_height
h_range=distance//die_width


contaminated = [[False]*cols for _ in range(rows)]

for i in range(rows):
    for j in range(cols):
        if raw_ink.iat[i, j] == bad_code:

            for dx in range(-v_range, v_range + 1):
                ni = i + dx
                if dx != 0 and 0 <= ni < rows:
                    if raw_ink.iat[ni, j] == good_code and matrix_df.iat[ni, j] == 1:
                        contaminated[ni][j] = True

            for dy in range(-h_range, h_range + 1):
                nj = j + dy
                if dy != 0 and 0 <= nj < cols:
                    if raw_ink.iat[i, nj] == good_code and matrix_df.iat[i, nj] == 1:
                        contaminated[i][nj] = True


good = bad = contam = 0
output_rows = []

for i in range(rows):
    row = []
    for j in range(cols):
        val = ink_df.iat[i, j]
        if val == "000":
            row.append("000")
        elif contaminated[i][j]:
            row.append("00C")
            contam += 1
        elif raw_ink.iat[i, j] == good_code:
            row.append("00G")
            good += 1
        elif raw_ink.iat[i, j] == bad_code or raw_ink.iat[i, j] == inactive_code:
            row.append("00B")
            bad += 1
        else:
            row.append("000")
    output_rows.append("ROWDATA: " + " ".join(row))

# Write to file
output_file = "M2T1.txt"
with open(output_file, "w") as f:
    f.write(f"NO OF GOOD DIES:{good}\n")
    f.write(f"NO OF BAD DIES:{bad}\n")
    f.write(f"NO OF CONTAMINATED DIES:{contam}\n")
    for row in output_rows:
        f.write(row + "\n")

print(f" Output  {output_file}")


✅ Milestone 2 complete. Output saved to: M2T1.txt


In [ ]:
# milestone_3.py
import pandas as pd
import numpy as np
from collections import deque

def rotate_back_to_0(df, orientation):
    arr = df.values
    if orientation == 90:
        arr = np.rot90(arr, k=3)
    elif orientation == 180:
        arr = np.rot90(arr, k=2)
    elif orientation == 270:
        arr = np.rot90(arr, k=1)
    return pd.DataFrame(arr)

def milestone_3(matrix_file, ink_file, output_file):
    matrix_df = pd.read_csv(matrix_file, header=None).astype(int)
    orientation = 0
    rowdata = []

    with open(ink_file, "r") as file:
        for line in file:
            if line.startswith("ORIENTATION:"):
                orientation = int(line.split(":")[1].strip())
            elif line.startswith("ROWDATA:"):
                rowdata.append(line.strip().replace("ROWDATA:", "").split())

    ink_df = pd.DataFrame(rowdata)
    ink_df = rotate_back_to_0(ink_df, orientation)

    rows, cols = min(matrix_df.shape[0], ink_df.shape[0]), min(matrix_df.shape[1], ink_df.shape[1])
    matrix_df, ink_df = matrix_df.iloc[:rows, :cols], ink_df.iloc[:rows, :cols]

    for i in range(rows):
        for j in range(cols):
            if matrix_df.iat[i, j] == 0:
                ink_df.iat[i, j] = "000"

    visited = [[False]*cols for _ in range(rows)]
    def is_bad(val): return val in ["060", "000"]

    def bfs(i, j):
        q = deque([(i, j)])
        visited[i][j] = True
        size = 1
        while q:
            x, y = q.popleft()
            for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
                nx, ny = x+dx, y+dy
                if 0 <= nx < rows and 0 <= ny < cols and not visited[nx][ny] and is_bad(ink_df.iat[nx, ny]):
                    visited[nx][ny] = True
                    q.append((nx, ny))
                    size += 1
        return size

    num_clusters = 0
    largest = 0
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j] and is_bad(ink_df.iat[i, j]):
                size = bfs(i, j)
                num_clusters += 1
                largest = max(largest, size)

    with open(output_file, "w") as f:
        f.write(f"{num_clusters},{largest}\n")

    print("✅ Milestone 3 (clusters) done:", output_file)


In [ ]:
import pandas as pd
import numpy as np

# 🔄 Rotate ink_df to 0° orientation
def rotate_back_to_0(df, orientation):
    arr = df.values
    if orientation == 90:
        arr = np.rot90(arr, k=3)  # 90° clockwise
    elif orientation == 180:
        arr = np.rot90(arr, k=2)  # 180°
    elif orientation == 270:
        arr = np.rot90(arr, k=1)  # 90° counter-clockwise
    return pd.DataFrame(arr)

# 🧠 Main Milestone 2 Function
def milestone_2(matrix_file, ink_file, output_file="M2T1.txt"):
    # Step 1: Load Matrix
    matrix_df = pd.read_csv(matrix_file, header=None).astype(int)

    # Step 2: Load Ink File + Parameters
    orientation = 0
    distance = 1
    die_height = 1
    die_width = 1
    good_code = bad_code = inactive_code = "100"
    rowdata = []

    with open(ink_file, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("ORIENTATION:"):
                orientation = int(line.split(":")[1].strip())
            elif line.startswith("DISTANCE:"):
                distance = int(line.split(":")[1].strip())
            elif line.startswith("DIESIZE:"):
                parts = line.split(":")[1].strip().split(",")
                die_height = int(parts[0])
                die_width = int(parts[1])
            elif line.startswith("GOOD_DIE:"):
                good_code = line.split(":")[1].strip()
            elif line.startswith("ACTIVE_BAD_DIE:"):
                bad_code = line.split(":")[1].strip()
            elif line.startswith("INACTIVE_BAD_DIE:"):
                inactive_code = line.split(":")[1].strip()
            elif line.startswith("ROWDATA:"):
                values = line.replace("ROWDATA:", "").split()
                rowdata.append(values)

    # Step 3: Rotate and align ink_df
    ink_df = pd.DataFrame(rowdata)
    ink_df = rotate_back_to_0(ink_df, orientation)

    rows, cols = min(matrix_df.shape[0], ink_df.shape[0]), min(matrix_df.shape[1], ink_df.shape[1])
    matrix_df = matrix_df.iloc[:rows, :cols].reset_index(drop=True)
    ink_df = ink_df.iloc[:rows, :cols].reset_index(drop=True)

    # Step 4: Invalidate dies based on matrix
    for i in range(rows):
        for j in range(cols):
            if matrix_df.iat[i, j] == 0:
                ink_df.iat[i, j] = "000"

    # Step 5: Convert DISTANCE to matrix steps
    vertical_range = distance // die_height
    horizontal_range = distance // die_width

    # Step 6: Create contamination matrix
    contaminated = [[False]*cols for _ in range(rows)]

    for i in range(rows):
        for j in range(cols):
            if ink_df.iat[i, j] == bad_code:
                # Vertical contamination (up/down)
                for dx in range(-vertical_range, vertical_range + 1):
                    ni = i + dx
                    if dx != 0 and 0 <= ni < rows and ink_df.iat[ni, j] == good_code:
                        contaminated[ni][j] = True
                # Horizontal contamination (left/right)
                for dy in range(-horizontal_range, horizontal_range + 1):
                    nj = j + dy
                    if dy != 0 and 0 <= nj < cols and ink_df.iat[i, nj] == good_code:
                        contaminated[i][nj] = True

    # Step 7: Generate formatted output
    good = bad = contam = 0
    output_rows = []

    for i in range(rows):
        row = []
        for j in range(cols):
            val = ink_df.iat[i, j]
            if val == "000":
                row.append("000")
            elif contaminated[i][j]:
                row.append("00C")
                contam += 1
            elif val == good_code:
                row.append("00G")
                good += 1
            elif val == bad_code or val == inactive_code:
                row.append("00B")
                bad += 1
            else:
                row.append("000")
        output_rows.append("ROWDATA: " + " ".join(row))

    # Step 8: Write to output file (visible in Colab)
    output_file = f"/content/{output_file}"
    with open(output_file, "w") as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{contam}\n")
        for line in output_rows:
            f.write(line + "\n")

    print("✅ Milestone 2 complete. Output saved to:", output_file)

    # Preview file content in cell
    with open(output_file, "r") as f:
        print("\n--- Output Preview ---\n")
        print(f.read())

    # Trigger download if in Colab
    try:
        from google.colab import files
        files.download(output_file)
    except ImportError:
        pass  # Ignore outside Colab


In [8]:
import csv
import math

def read_matrix(file_path):
    with open(file_path) as f:
        return [list(map(int, line.strip().split(','))) for line in f]

def parse_input(file_path):
    data = {}
    ink = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith("ROWDATA:"):
                values = line.replace("ROWDATA:", "").strip().split()
                ink.append(values)
            elif ":" in line:
                key, value = line.split(":")
                key = key.strip().upper()
                if key == "DIESIZE":
                    data["DIESIZE"] = tuple(map(int, value.split(',')))
                elif key in ["DISTANCE", "ORIENTATION"]:
                    data[key] = int(value.strip())
                else:
                    data[key] = value.strip()
    return data, ink

def rotate_ink(ink, orientation):
    if orientation == 90:
        return [list(row) for row in zip(*ink[::-1])]
    return ink

def align_to_center(matrix, ink):
    m_rows, m_cols = len(matrix), len(matrix[0])
    i_rows, i_cols = len(ink), len(ink[0])
    start_row = (m_rows - i_rows) // 2
    start_col = (m_cols - i_cols) // 2
    aligned = [["" for _ in range(m_cols)] for _ in range(m_rows)]
    for i in range(i_rows):
        for j in range(i_cols):
            aligned[start_row + i][start_col + j] = ink[i][j]
    return aligned

def mark_invalid_dies(matrix, ink):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 0:
                ink[i][j] = "000"
    return ink

def mark_contamination(ink, die_height, die_width, distance):
    rows, cols = len(ink), len(ink[0])
    contaminated = set()

    spread_rows = math.floor(distance / die_height)
    spread_cols = math.floor(distance / die_width)

    for i in range(rows):
        for j in range(cols):
            if ink[i][j] == "060":  # active bad die
                for dx in range(-spread_rows, spread_rows + 1):
                    for dy in range(-spread_cols, spread_cols + 1):
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < rows and 0 <= nj < cols:
                            if ink[ni][nj] == "100":
                                contaminated.add((ni, nj))

    for i, j in contaminated:
        ink[i][j] = "00C"
    return ink

def convert_labels(ink):
    good, bad, contaminated = 0, 0, 0
    output = []

    for row in ink:
        new_row = []
        for val in row:
            if val == "100":
                new_row.append("00G")
                good += 1
            elif val == "060" or val == "030":
                new_row.append("00B")
                bad += 1
            elif val == "00C":
                new_row.append("00C")
                contaminated += 1
            elif val == "000":
                new_row.append("000")
            else:
                new_row.append("000")
        output.append(new_row)

    return good, bad, contaminated, output

def write_output(file_name, good, bad, contaminated, output_matrix):
    with open(file_name, 'w') as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{contaminated}\n")
        for row in output_matrix:
            f.write("ROWDATA:" + ' '.join(row) + '\n')

# === MAIN ===

matrix = read_matrix("MATRIX.csv")
data, ink_raw = parse_input("INPUT.txt")

rotated_ink = rotate_ink(ink_raw, data["ORIENTATION"])
aligned_ink = align_to_center(matrix, rotated_ink)
invalid_marked = mark_invalid_dies(matrix, aligned_ink)
contaminated_ink = mark_contamination(invalid_marked, *data["DIESIZE"], data["DISTANCE"])
good, bad, contaminated, output_matrix = convert_labels(contaminated_ink)

write_output("M2T1.txt", good, bad, contaminated, output_matrix)


In [11]:
###neww
import math

def read_matrix(file_path):
    with open(file_path) as f:
        return [list(map(int, line.strip().split(','))) for line in f]

def parse_input(file_path):
    data = {}
    ink = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith("ROWDATA:"):
                ink.append(line.replace("ROWDATA:", "").strip().split())
            elif ":" in line:
                key, value = line.split(":")
                key = key.strip().upper()
                if key == "DIESIZE":
                    data["DIESIZE"] = tuple(map(int, value.split(',')))
                elif key == "DISTANCE" or key == "ORIENTATION":
                    data[key] = int(value.strip())
                else:
                    data[key] = value.strip()
    return data, ink

def rotate_ink(ink, orientation):
    if orientation == 90:
        return [list(row) for row in zip(*ink[::-1])]
    return ink

def align_to_center(matrix, ink):
    m_rows, m_cols = len(matrix), len(matrix[0])
    i_rows, i_cols = len(ink), len(ink[0])
    aligned = [["" for _ in range(m_cols)] for _ in range(m_rows)]
    row_shift = (m_rows - i_rows) // 2
    col_shift = (m_cols - i_cols) // 2
    for i in range(i_rows):
        for j in range(i_cols):
            aligned[i + row_shift][j + col_shift] = ink[i][j]
    return aligned

def mark_invalid_dies(matrix, ink, invalid_val):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 0:
                ink[i][j] = invalid_val
    return ink

def mark_contamination(ink, data):
    die_h, die_w = data["DIESIZE"]
    distance = data["DISTANCE"]
    active_bad = data["ACTIVE_BAD_DIE"]
    good_val = data["GOOD_DIE"]
    rows, cols = len(ink), len(ink[0])

    spread_rows = math.floor(distance / die_h)
    spread_cols = math.floor(distance / die_w)

    contaminated = set()

    for i in range(rows):
        for j in range(cols):
            if ink[i][j] == active_bad:
                for dx in range(-spread_rows, spread_rows + 1):
                    for dy in range(-spread_cols, spread_cols + 1):
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < rows and 0 <= nj < cols:
                            if ink[ni][nj] == good_val:
                                contaminated.add((ni, nj))

    for i, j in contaminated:
        ink[i][j] = "00C"
    return ink

def convert_labels(ink, data):
    good_count = bad_count = cont_count = 0
    good_val = data["GOOD_DIE"]
    bad_vals = [data["ACTIVE_BAD_DIE"], data["INACTIVE_BAD_DIE"]]
    invalid_val = data["INVALID_DIE"]
    result = []

    for row in ink:
        new_row = []
        for val in row:
            if val == "00C":
                new_row.append("00C")
                cont_count += 1
            elif val == good_val:
                new_row.append("00G")
                good_count += 1
            elif val in bad_vals:
                new_row.append("00B")
                bad_count += 1
            elif val == invalid_val or val == "":
                new_row.append("000")
            else:
                new_row.append("000")
        result.append(new_row)
    return good_count, bad_count, cont_count, result

def write_output(file_name, good, bad, cont, output_matrix):
    with open(file_name, 'w') as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{cont}\n")
        for row in output_matrix:
            f.write("ROWDATA:" + ' '.join(row) + '\n')


matrix = read_matrix("MATRIX.csv")
data, ink_raw = parse_input("INPUT.txt")

rotated_ink = rotate_ink(ink_raw, data["ORIENTATION"])
aligned_ink = align_to_center(matrix, rotated_ink)
marked_invalid = mark_invalid_dies(matrix, aligned_ink, data["INVALID_DIE"])
marked_contamination = mark_contamination(marked_invalid, data)
good, bad, cont, output_matrix = convert_labels(marked_contamination, data)

write_output("M2T2.txt", good, bad, cont, output_matrix)


In [26]:
##neww22

import math
def read_matrix(file_path):
    matrix = []
    with open(file_path) as f:
        for line in f:
            row = []
            # Split by comma and strip whitespace
            values = line.strip().split(',')
            for val in values:
                try:
                    # Try to convert to integer
                    row.append(int(val))
                except ValueError:
                    # If conversion fails, skip the value
                    pass
            if row: # Only append if the row is not empty after filtering
                matrix.append(row)
    return matrix


def parse_input(file_path):
    data = {}
    ink = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith("ROWDATA:"):
                ink.append(line.replace("ROWDATA:", "").strip().split())
            elif ":" in line:
                key, value = line.split(":")
                key = key.strip().upper()
                if key == "DIESIZE":
                    data["DIESIZE"] = tuple(map(int, value.split(',')))
                elif key in ["DISTANCE", "ORIENTATION"]:
                    data[key] = int(value.strip())
                else:
                    data[key] = value.strip()
    return data, ink

def rotate_ink(ink, orientation):
    if orientation == 90:
        return [list(row) for row in zip(*ink[::-1])]
    return ink

def align_to_center(matrix, ink):
    m_rows, m_cols = len(matrix), len(matrix[0])
    i_rows, i_cols = len(ink), len(ink[0])
    aligned = [["" for _ in range(m_cols)] for _ in range(m_rows)]
    row_shift = (m_rows - i_rows) // 2
    col_shift = (m_cols - i_cols) // 2
    for i in range(i_rows):
        for j in range(i_cols):
            aligned[row_shift + i][col_shift + j] = ink[i][j]
    return aligned

def mark_invalid_dies(matrix, ink, invalid_val):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 0:
                ink[i][j] = invalid_val
    return ink

def mark_contamination(ink, data):
    die_h, die_w = data["DIESIZE"]
    distance = data["DISTANCE"]
    active_bad = data["ACTIVE_BAD_DIE"]
    good_val = data["GOOD_DIE"]
    rows, cols = len(ink), len(ink[0])

    # ✅ Use ceil instead of floor (Validator expects even partial dies to be affected)
    spread_rows = math.ceil(distance / die_h)
    spread_cols = math.ceil(distance / die_w)

    contaminated = set()

    for i in range(rows):
        for j in range(cols):
            if ink[i][j] == active_bad:
                for dx in range(-spread_rows, spread_rows + 1):
                    for dy in range(-spread_cols, spread_cols + 1):
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < rows and 0 <= nj < cols:
                            if ink[ni][nj] == good_val:
                                contaminated.add((ni, nj))

    for i, j in contaminated:
        ink[i][j] = "00C"
    return ink

def convert_labels(ink, data):
    good_count = bad_count = cont_count = 0
    good_val = data["GOOD_DIE"]
    bad_vals = [data["ACTIVE_BAD_DIE"], data["INACTIVE_BAD_DIE"]]
    invalid_val = data["INVALID_DIE"]
    result = []

    for row in ink:
        new_row = []
        for val in row:
            if val == "00C":
                new_row.append("00C")
                cont_count += 1
            elif val == good_val:
                new_row.append("00G")
                good_count += 1
            elif val in bad_vals:
                new_row.append("00B")
                bad_count += 1
            elif val == invalid_val or val == "":
                new_row.append("000")
            else:
                new_row.append("000")
        result.append(new_row)
    return good_count, bad_count, cont_count, result
4t
def write_output(file_name, good, bad, cont, output_matrix):
    with open(file_name, 'w') as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{cont}\n")
        for row in output_matrix:
            f.write("ROWDATA:" + ' '.join(row) + '\n')


matrix = read_matrix("MATRIX.csv")
data, ink_raw = parse_input("INPUT.txt")

rotated_ink = rotate_ink(ink_raw, data["ORIENTATION"])
aligned_ink = align_to_center(matrix, rotated_ink)
marked_invalid = mark_invalid_dies(matrix, aligned_ink, data["INVALID_DIE"])
marked_contamination = mark_contamination(marked_invalid, data)
good, bad, cont, output_matrix = convert_labels(marked_contamination, data)

write_output("M2T8.txt", good, bad, cont, output_matrix)

In [27]:
import csv
import numpy as np
import math
import re

def load_matrix(file):
    with open(file, 'r') as f:
        reader = csv.reader(f)
        return [
            [int(cell.strip()) for cell in row if cell.strip() != '']
            for row in reader
        ]

def load_input(file):
    meta, data, testcase = {}, [], None
    with open(file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("NAME:"):
                testcase = re.search(r'TESTCASE(\d+)', line).group(1)
            elif line.startswith("ROWDATA:"):
                data.append(line.replace("ROWDATA:", "").split())
            elif ':' in line:
                k, v = line.split(":", 1)
                meta[k.strip()] = v.strip()
    print(meta)
    return int(testcase), meta, data

def rotate_to_0(grid, orientation):
  if 0 <= row_shift + i < m_rows and 0 <= col_shift + j < m_cols:

    if orientation == 0: return grid
    array = np.array(grid)
    if orientation == 90: return np.rot90(array, k=3).tolist()
    if orientation == 180: return np.rot90(array, k=2).tolist()
    if orientation == 270: return np.rot90(array, k=1).tolist()
    raise ValueError("Invalid orientation")


def center_align(matrix, ink):
    m, n = len(matrix), len(matrix[0])
    h, w = len(ink), len(ink[0])
    m_mid=m//2 if m%2==0 else m//2
    i_mid=h//2 if h%2==0 else h//2
    top=max(0,m_mid-i_mid)
    left=max(0,n//2-(w//2))
    full = [['000'] * n for _ in range(m)]
    for i in range(h):
        for j in range(w):
            if top+i<m and left+j<n:
                full[top+i][left+j]=ink[i][j]
    return full

def apply_mask(grid, matrix, invalid_code):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if matrix[i][j] == 0:
                grid[i][j] = invalid_code
    return grid


def replace_die_codes(grid, good, active_bad, inactive_bad):
    active_positions = []

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == good:
                grid[i][j] = '00G'
            elif grid[i][j] == active_bad:
                grid[i][j] = '00B'
                active_positions.append((i, j))
            elif grid[i][j] == inactive_bad:
                grid[i][j] = '00B'
            elif grid[i][j] == '000':
                continue
    return grid, active_positions


def mark_contaminated(grid, active_positions, die_h, die_w, dist):
    rows, cols = len(grid), len(grid[0])
    r_radius = math.ceil(dist / die_h)
    c_radius = math.ceil(dist / die_w)
    contaminated = 0
    contaminated_map = [[False]*cols for _ in range(rows)]

    for i, j in active_positions:
        for dx in range(-r_radius, r_radius+1):
            for dy in range(-c_radius, c_radius+1):
                ni, nj = i + dx, j + dy
                if 0 <= ni < rows and 0 <= nj < cols:
                    if grid[ni][nj] == '00G' and not contaminated_map[ni][nj]:
                        grid[ni][nj] = '00C'
                        contaminated_map[ni][nj] = True
                        contaminated += 1

    return grid, contaminated


# Count dies
def count_dies(grid):
    good = sum(row.count('00G') for row in grid)
    bad = sum(row.count('00B') for row in grid)
    return good, bad


def write_output(filename, good, bad, contaminated, grid):
    with open(filename, 'w') as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{contaminated}\n")
        for row in grid:
            f.write("ROWDATA:" + ' '.join(row) + '\n')


matrix = load_matrix("MATRIX.csv")
testcase, meta, ink = load_input("INPUT.txt")
orientation = int(meta["ORIENTATION"])
die_h, die_w = map(int, meta["DIESIZE"].split(","))
distance = int(meta["DISTANCE"])
good = meta["GOOD_DIE"]
active_bad = meta["ACTIVE_BAD_DIE"]
inactive_bad = meta["INACTIVE_BAD_DIE"]

rotated = rotate_to_0(ink, orientation)
aligned = center_align(matrix, rotated)
masked = apply_mask(aligned, matrix, '000')

coded, active_positions = replace_die_codes(masked, good, active_bad, inactive_bad)
final_grid, contaminated = mark_contaminated(coded, active_positions, die_h, die_w, distance)

g_count, b_count = count_dies(final_grid)

output_file = f"M2T{testcase}.txt"
write_output(output_file, g_count, b_count, contaminated, final_grid)
print(f"Output {output_file}")

{'DIESIZE': '250,200', 'DISTANCE': '200', 'GOOD_DIE': '100', 'ACTIVE_BAD_DIE': '009', 'INACTIVE_BAD_DIE': '003', 'INVALID_DIE': '000', 'ORIENTATION': '90'}
Output M2T8.txt


In [29]:
#milestone 3

import math

def read_matrix(file_path):
    matrix = []
    with open(file_path) as f:
        for line in f:
            row = []
            values = line.strip().split(',')
            for val in values:
                try:
                    row.append(int(val))
                except ValueError:
                    pass
            if row:
                matrix.append(row)
    return matrix

def parse_input(file_path):
    data = {}
    ink = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith("ROWDATA:"):
                ink.append(line.replace("ROWDATA:", "").strip().split())
            elif ":" in line:
                key, value = line.split(":")
                key = key.strip().upper()
                if key == "DIESIZE":
                    data["DIESIZE"] = tuple(map(int, value.split(',')))
                elif key in ["DISTANCE", "ORIENTATION"]:
                    data[key] = int(value.strip())
                else:
                    data[key] = value.strip()
    return data, ink

def rotate_ink(ink, orientation):
    if orientation == 90:
        return [list(row) for row in zip(*ink[::-1])]
    return ink

def align_to_center(matrix, ink):
    m_rows, m_cols = len(matrix), len(matrix[0])
    i_rows, i_cols = len(ink), len(ink[0])
    aligned = [["" for _ in range(m_cols)] for _ in range(m_rows)]
    row_shift = (m_rows - i_rows) // 2
    col_shift = (m_cols - i_cols) // 2
    for i in range(i_rows):
        for j in range(i_cols):
            if 0 <= row_shift + i < m_rows and 0 <= col_shift + j < m_cols:
                aligned[row_shift + i][col_shift + j] = ink[i][j]
    return aligned

def mark_invalid_dies(matrix, ink, invalid_val):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 0:
                ink[i][j] = invalid_val
    return ink

def mark_contamination(ink, data):
    die_h, die_w = data["DIESIZE"]
    distance = data["DISTANCE"]
    active_bad = data["ACTIVE_BAD_DIE"]
    good_val = data["GOOD_DIE"]
    rows, cols = len(ink), len(ink[0])

    spread_rows = math.ceil(distance / die_h)
    spread_cols = math.ceil(distance / die_w)

    contaminated = set()

    for i in range(rows):
        for j in range(cols):
            if ink[i][j] == active_bad:
                for dx in range(-spread_rows, spread_rows + 1):
                    for dy in range(-spread_cols, spread_cols + 1):
                        ni, nj = i + dx, j + dy
                        if 0 <= ni < rows and 0 <= nj < cols:
                            if ink[ni][nj] == good_val:
                                contaminated.add((ni, nj))

    for i, j in contaminated:
        ink[i][j] = "00C"
    return ink

def mark_edge_clusters(ink, data):
    rows, cols = len(ink), len(ink[0])
    visited = [[False]*cols for _ in range(rows)]

    def dfs(i, j, cluster):
        stack = [(i, j)]
        is_edge = False
        while stack:
            x, y = stack.pop()
            if visited[x][y]:
                continue
            visited[x][y] = True
            cluster.append((x, y))
            if x == 0 or x == rows-1 or y == 0 or y == cols-1:
                is_edge = True
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                ni, nj = x+dx, y+dy
                if 0 <= ni < rows and 0 <= nj < cols:
                    if not visited[ni][nj] and ink[ni][nj] == "00B":
                        stack.append((ni, nj))
        return is_edge

    for i in range(rows):
        for j in range(cols):
            if ink[i][j] == "00B" and not visited[i][j]:
                cluster = []
                touches_edge = dfs(i, j, cluster)
                if touches_edge:
                    for x, y in cluster:
                        ink[x][y] = "00E"
    return ink

def convert_labels(ink, data):
    good_count = bad_count = cont_count = 0
    good_val = data["GOOD_DIE"]
    bad_vals = [data["ACTIVE_BAD_DIE"], data["INACTIVE_BAD_DIE"]]
    invalid_val = data["INVALID_DIE"]
    result = []

    for row in ink:
        new_row = []
        for val in row:
            if val == "00C":
                new_row.append("00C")
                cont_count += 1
            elif val == "00E":
                new_row.append("00E")
                bad_count += 1
            elif val == good_val:
                new_row.append("00G")
                good_count += 1
            elif val in bad_vals:
                new_row.append("00B")
                bad_count += 1
            elif val == invalid_val or val == "":
                new_row.append("000")
            else:
                new_row.append("000")
        result.append(new_row)
    return good_count, bad_count, cont_count, result

def write_output(file_name, good, bad, cont, output_matrix):
    with open(file_name, 'w') as f:
        f.write(f"NO OF GOOD DIES:{good}\n")
        f.write(f"NO OF BAD DIES:{bad}\n")
        f.write(f"NO OF CONTAMINATED DIES:{cont}\n")
        for row in output_matrix:
            f.write("ROWDATA:" + ' '.join(row) + '\n')

# Example execution (you can plug your files below):
# matrix = read_matrix("MATRIX.csv")
# data, ink_raw = parse_input("INPUT.txt")
# rotated_ink = rotate_ink(ink_raw, data["ORIENTATION"])
# aligned_ink = align_to_center(matrix, rotated_ink)
# marked_invalid = mark_invalid_dies(matrix, aligned_ink, data["INVALID_DIE"])
# marked_contamination = mark_contamination(marked_invalid, data)
# marked_edge_clusters = mark_edge_clusters(marked_contamination, data)
# good, bad, cont, output_matrix = convert_labels(marked_edge_clusters, data)
# write_output("M3T8.txt", good, bad, cont, output_matrix)
